In [5]:

import requests
from bs4 import BeautifulSoup
import json
import time


In [6]:
def scrape_freightbook(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        soup = BeautifulSoup(response.content, 'html.parser')
        search_results = soup.find('div', class_='search-results')

        if not search_results:
            print("Warning: Could not find 'search-results' div.  Page structure may have changed.")
            return []

        business_cards = search_results.find_all('div', class_='business-card vcard')
        
        data = []
        for card in business_cards:
            org_div = card.find('div', class_='org')
            if org_div:
                organization = org_div.find('h2').text.strip()

                address_div = card.find('div', class_='street-address')
                address = address_div.text.strip() if address_div else "Address not found"

                country_div = card.find('div', class_='country-name')
                country = country_div.text.strip() if country_div else "Country not found"

                data.append({
                    'organization': organization,
                    'address': address,
                    'country': country
                })
            else:
                print("Warning: Could not find 'org' div in a business card.")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

In [9]:
def main():
    url = "https://m.freightbook.net/member/results?country=154"
    scraped_data = scrape_freightbook(url)

    if scraped_data:
        print(f"Successfully scraped {len(scraped_data)} records.")
        with open('freightbook_data_nepal.json', 'w', encoding='utf-8') as f:
            json.dump(scraped_data, f, indent=4, ensure_ascii=False)  # Ensure proper encoding
        print("Data saved to freightbook_data_nepal.json")
    else:
        print("No data scraped.")

In [10]:
if __name__ == "__main__":
    main()

Successfully scraped 1 records.
Data saved to freightbook_data_nepal.json
